Coleta de Noticias

In [1]:
## Fontes de Notícias

fonte_g1 = "https://g1.globo.com/economia/dolar/"
fonte_cnn = "https://www.cnnbrasil.com.br/tudo-sobre/dolar/"
fonte_folha = "https://www1.folha.uol.com.br/folha-topicos/dolar/"

In [2]:
## Importando Bibliotecas para Web Scraping
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from datetime import datetime
import time
import selenium.webdriver as webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException



In [3]:
## Extraindo Notícias do G1 com o Selenium
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
driver.get(fonte_g1)
time.sleep(5)  # Aguarda o carregamento da página
driver.find_element(By.CSS_SELECTOR, "svg.fc-cancel-icon-svg").click() # Fecha o pop-up de ADS

# Scroll para carregar mais notícias
for _ in range(10):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    
html_g1 = driver.page_source


driver.quit()


There was an error managing chromedriver (error decoding response body); using driver found in the cache


In [4]:

soup = BeautifulSoup(html_g1, "html.parser")

# Tratando os dados do G1
url_imagem = [img["src"] for img in soup.select("img.bstn-fd-picture-image")]
titulos = [t.get_text(strip=True) for t in soup.select("div.feed-post-body-title a")]
links = [a["href"] for a in soup.select("a.feed-post-link")]
data_publicacao = [
    f"{link.split('/')[7]}/{link.split('/')[6]}/{link.split('/')[5]}"
    for link in links
]

# Salvando os dados em um DataFrame com a data de extração
data_extracao = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
df_g1 = pd.DataFrame({
    "urlImagem": url_imagem,
    "dataPublicacao": data_publicacao,
    "titulo": titulos,
    "link": links,
    "fonte": "G1",
    "dataExtracao": data_extracao
})

df_noticias = df_g1.copy()

In [5]:
## Extraindo Notícias do CNN 
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
driver.get(fonte_cnn)
time.sleep(5)  # Aguarda o carregamento da página

wait = WebDriverWait(driver, 10)
botao_ok = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn-agree button")))

# Clica no botão
botao_ok.click()


for i in range(2):  # Navega por 2 páginas
    time.sleep(5)
    html_cnn = driver.page_source
    soup = BeautifulSoup(html_cnn, "html.parser")
    container = soup.select_one('ul[data-section="article_list"]')
    

    # Printa a pagina percorrida
    url_imagem= []
    if container:
        url_imagem = [img["src"] for img in container.select("img") if img.get("src")]
    titulos = [t.get_text(strip=True) for t in soup.select("div.flex.flex-col.gap-4 h2")]
    links = [a["href"] for a in soup.select("div.flex.flex-col.gap-4 a")]
    links = [item for item in links if item != 'https://www.cnnbrasil.com.br/ao-vivo/']
    data_publicacao = []
    for tag in soup.select("time"):
        raw = tag.get("datetime")
        if raw:
            data_publicacao.append(datetime.fromisoformat(raw).strftime("%d/%m/%Y"))

    data_extracao = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    df_cnn = pd.DataFrame({
        "urlImagem": url_imagem,
        "dataPublicacao": data_publicacao,
        "titulo": titulos,
        "link": links,
        "fonte": "CNN",
        "dataExtracao": data_extracao
    })

    df_noticias = pd.concat([df_noticias, df_cnn], ignore_index=True)

    driver.find_element(By.CSS_SELECTOR, "a[aria-label='Ir para próxima página']").click()    
    
    time.sleep(5)

driver.quit()


In [6]:


options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

driver.get(fonte_folha)

wait = WebDriverWait(driver, 20)

for i in range(4):
    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "main .row, main article, main li")))

        last_height = driver.execute_script("return document.body.scrollHeight")
        found = False

        for _ in range(50):  
            candidatos = driver.find_elements(By.CSS_SELECTOR, "button.c-button.c-button--expand")
        
            if candidatos:
                botao = candidatos[0]
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", botao)
                time.sleep(0.8)
                try:
                    wait.until(EC.element_to_be_clickable((By.XPATH, ".//button[contains(@class,'c-button') or @data-pagination-trigger]")))
                except TimeoutException:
                    pass 

                try:
                    botao.click()
                except Exception:
                    driver.execute_script("arguments[0].click();", botao)
                break
    finally:
        time.sleep(2)

html_folha = driver.page_source       
driver.quit()


In [7]:
soup = BeautifulSoup(html_folha, "html.parser") 

url_imagem = [img.get("data-src") for img in soup.select("img.c-headline__image") if img.get("data-src")]
titulos = [t.get_text(strip=True) for t in soup.select("h2.c-headline__title")]  
data_publicacao = [t.get("datetime") for t in soup.select("time.c-headline__dateline[itemprop='datePublished']")]
links = [a["href"] for a in soup.select("a.c-headline__url")]

# Converter a dataPublicacao para dd/mm/yyyy
data_publicacao = [
    datetime.strptime(d, "%Y-%m-%d %H:%M:%S").strftime("%d/%m/%Y")
    for d in data_publicacao if d
]

def pad(lst, n, fill=""):
    return lst + [fill] * (n - len(lst))

n = max(len(url_imagem), len(data_publicacao), len(titulos), len(links))

df_folha = pd.DataFrame({
    "urlImagem":     pad(url_imagem, n, ""),
    "dataPublicacao":pad(data_publicacao, n, ""),
    "titulo":        pad(titulos, n, ""),
    "link":          pad(links, n, ""),
    "fonte":         ["Folha"] * n,
    "dataExtracao":  [datetime.now().strftime("%d/%m/%Y %H:%M:%S")] * n,
})

df_folha = df_folha[df_folha['urlImagem'] != '']

df_noticias = pd.concat([df_noticias, df_folha], ignore_index=True)


In [8]:
df_noticias

,urlImagem,dataPublicacao,titulo,link,fonte,dataExtracao
0,https://s2-g1.glbimg.com/5WrVoTrL2c24Do4OEg3tD...,24/10/2025,"Dólar tem leve alta e fecha a R$ 5,39 com dado...",https://g1.globo.com/economia/noticia/2025/10/...,G1,25/10/2025 13:25:01
1,https://s2-g1.glbimg.com/B2YXPBxijNajZUUdb5491...,23/10/2025,"Dólar cai e fecha a R$ 5,38 após sanção dos EU...",https://g1.globo.com/economia/noticia/2025/10/...,G1,25/10/2025 13:25:01
2,https://s2-g1.glbimg.com/Bf5ck8uP4djHFeG95zeBz...,22/10/2025,"Dólar tem leve alta e fecha a R$ 5,39 com expe...",https://g1.globo.com/economia/noticia/2025/10/...,G1,25/10/2025 13:25:01
3,https://s2-g1.glbimg.com/GNsmi6t9Kge-50Km_oq7Y...,21/10/2025,"Dólar sobe e fecha em R$ 5,39, com expectativa...",https://g1.globo.com/economia/noticia/2025/10/...,G1,25/10/2025 13:25:01
4,https://s2-g1.glbimg.com/GNsmi6t9Kge-50Km_oq7Y...,20/10/2025,"Dólar cai e fecha em R$ 5,37, com atenção ao m...",https://g1.globo.com/economia/noticia/2025/10/...,G1,25/10/2025 13:25:01
...,...,...,...,...,...,...
146,https://f.i.uol.com.br/fotografia/2025/09/17/1...,17/09/2025,"O real valorizou, mas o risco fiscal persiste",https://www1.folha.uol.com.br/colunas/solange-...,Folha,25/10/2025 13:27:25
147,https://f.i.uol.com.br/fotografia/2025/09/17/1...,17/09/2025,"Com divisões internas, Fed faz primeiro corte ...",https://www1.folha.uol.com.br/mercado/2025/09/...,Folha,25/10/2025 13:27:25
148,https://f.i.uol.com.br/fotografia/2025/01/08/1...,17/09/2025,Dólar fecha quase estável e Bolsa sobe após co...,https://www1.folha.uol.com.br/mercado/2025/09/...,Folha,25/10/2025 13:27:25
149,https://f.i.uol.com.br/fotografia/2025/05/30/1...,17/09/2025,A conta chega em 2027,https://www1.folha.uol.com.br/opiniao/2025/09/...,Folha,25/10/2025 13:27:25


In [9]:
!pip install duckdb


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\Anderson\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import duckdb

In [ ]:
df = df_noticias.copy()

for col in ["dataPublicacao", "dataExtracao"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], dayfirst=True, errors="coerce")

df["hash"] = (df["titulo"] + df["fonte"]).apply(lambda x: hash(x))
df = df.drop_duplicates(subset=["hash"])

In [40]:
DB_PATH = "dados_dolar.duckdb"   
TABLE   = "noticias"   

con = duckdb.connect(DB_PATH)

# Cria a tabela se não existir (sem dados)
con.execute(f"""
CREATE TABLE IF NOT EXISTS {TABLE} (
    urlImagem       TEXT,
    dataPublicacao  TIMESTAMP,
    titulo          TEXT,
    link            TEXT,
    fonte           TEXT,
    dataExtracao    TIMESTAMP,
    hash            TEXT, 
);
""")


In [35]:
# dropar tabela antiga
con = duckdb.connect(DB_PATH)
con.execute(f"DROP TABLE IF EXISTS {TABLE};")


In [42]:
con = duckdb.connect(DB_PATH)
print(con.execute(f"SELECT COUNT(*) FROM {TABLE};").fetchone())
print(con.execute(f"SELECT * FROM {TABLE} ORDER BY dataPublicacao DESC NULLS LAST LIMIT 5;").fetchdf())
con.execute("""
COPY noticias TO './exports/noticias.csv' (HEADER, DELIMITER ',');
""")

con.close()


(151,)
                                           urlImagem dataPublicacao  \
0  https://s2-g1.glbimg.com/5WrVoTrL2c24Do4OEg3tD...     2025-10-24   
1  https://admin.cnnbrasil.com.br/wp-content/uplo...     2025-10-24   
2  https://f.i.uol.com.br/fotografia/2025/06/02/1...     2025-10-24   
3  https://f.i.uol.com.br/fotografia/2025/02/12/1...     2025-10-24   
4  https://f.i.uol.com.br/fotografia/2019/01/31/1...     2025-10-24   

                                              titulo  \
0  Dólar tem leve alta e fecha a R$ 5,39 com dado...   
1  Ibovespa sobe 1,9% na semana após dados de inf...   
2  Empréstimos em yuan no exterior disparam com c...   
3  Inflação dos EUA sobe menos que o esperado em ...   
4  Dólar e Bolsa fecham em leve alta após dados d...   

                                                link  fonte  \
0  https://g1.globo.com/economia/noticia/2025/10/...     G1   
1  https://www.cnnbrasil.com.br/economia/mercado/...    CNN   
2  https://www1.folha.uol.com.br/mercado